<a href="https://colab.research.google.com/github/nshamid/HO2_MLOps_NabilahShamid/blob/main/HO2_MLOps_NabilahShamid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Source

In [38]:
# Import libraries
import requests
import zipfile
import os
import pandas as pd
import numpy as np

In [39]:
# Dataset yang bersih diunggah di Github untuk kemudahan akses
dataset_url = 'https://github.com/nshamid/HO2_MLOps_NabilahShamid/raw/refs/heads/main/cleaned_job_data.zip'

# Unduh file ZIP
response = requests.get(dataset_url)
with open('dataset.zip', 'wb') as f:
    f.write(response.content)

print("ZIP file berhasil diunduh!")

ZIP file berhasil diunduh!


In [40]:
# Mengecek nama file
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    file_list = zip_ref.namelist()
    print("Isi file di dalam ZIP:")
    for f in file_list:
        print(f)

    # Ekstrak semua file
    zip_ref.extractall()

print("ZIP file berhasil diekstrak!")

Isi file di dalam ZIP:
cleaned_job_data.csv
ZIP file berhasil diekstrak!


In [41]:
file_name = 'cleaned_job_data.csv'

# Load CSV ke DataFrame
df = pd.read_csv(file_name)

In [42]:
print(df.head())

     job_id                            job_title  \
0  72761527                        data engineer   
1  72787241         machine learning engineer ai   
2  72866732              senior riskdata analyst   
3  72851872  senior data engineer hybrid working   
4  72526811        data scientist hybrid working   

                             company  \
0          anhsin technology sdn bhd   
1            accordia global sdn bhd   
2  toyota capital malaysia sdn. bhd.   
3                               seek   
4              seek asia (jobstreet)   

                                        descriptions          location  \
0  design develop and maintain scalable and robus...      kuala lumpur   
1  design develop and deploy machine learning mod...  shah alam/subang   
2  analyse data to better understand potential ri...          petaling   
3  design development testing and operation of bi...      kuala lumpur   
4  research build deploy and maintain recommendat...      kuala lumpur   

 

# Hands-On 2
- Vektorisasi teks.
- Labelling & Similarity Mapping.
- Recommendation Modelling.

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Kolom yang akan ditindak lanjuti
- `leaned_job_title`
- `cleaned_descriptions`

In [44]:
df[['job_title', 'descriptions']].head()

,job_title,descriptions
0,data engineer,design develop and maintain scalable and robus...
1,machine learning engineer ai,design develop and deploy machine learning mod...
2,senior riskdata analyst,analyse data to better understand potential ri...
3,senior data engineer hybrid working,design development testing and operation of bi...
4,data scientist hybrid working,research build deploy and maintain recommendat...


In [45]:
# Tangani nilai yang hilang di kolom 'descriptions' dan 'job_title' dengan mengisi string kosong.
df['descriptions'] = df['descriptions'].fillna('')
df['job_title'] = df['job_title'].fillna('')

# Membuat kolom 'combined_text' dengan menggabungkan konten dari 'job_title' dan 'descriptions'.
df['combined_text'] = df['job_title'] + ' ' + df['descriptions']
print("Kolom 'combined_text' berhasil dibuat dengan menggabungkan 'job_title' dan 'descriptions' yang sudah bersih.")
print(df[['job_title', 'descriptions', 'combined_text']].head())

Kolom 'combined_text' berhasil dibuat dengan menggabungkan 'job_title' dan 'descriptions' yang sudah bersih.
                             job_title  \
0                        data engineer   
1         machine learning engineer ai   
2              senior riskdata analyst   
3  senior data engineer hybrid working   
4        data scientist hybrid working   

                                        descriptions  \
0  design develop and maintain scalable and robus...   
1  design develop and deploy machine learning mod...   
2  analyse data to better understand potential ri...   
3  design development testing and operation of bi...   
4  research build deploy and maintain recommendat...   

                                       combined_text  
0  data engineer design develop and maintain scal...  
1  machine learning engineer ai design develop an...  
2  senior riskdata analyst analyse data to better...  
3  senior data engineer hybrid working design dev...  
4  data scientist hybrid w

## TF-IDF Vectorization

In [46]:
# max_features=5000 membatasi jumlah fitur (kata/term unik) yang akan dipertimbangkan.
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit dan transform kolom 'combined_text' untuk mendapatkan TF-IDF matrix.
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_text'])

print("TF-IDF Vectorization Selesai pada kolom 'combined_text'.")
print(f"Shape dari TF-IDF matrix: {tfidf_matrix.shape}")

TF-IDF Vectorization Selesai pada kolom 'combined_text'.
Shape dari TF-IDF matrix: (606, 2910)


## Labelling & Similarity Mapping (Cosine Similarity)

In [47]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("\nCosine Similarity Matrix Selesai Dihitung (berdasarkan 'combined_text').")
print(f"Shape dari Cosine Similarity Matrix: {cosine_sim_matrix.shape}")

# Membuat peta (mapping) dari judul pekerjaan yang sudah bersih ke indeks DataFrame-nya.
indices = pd.Series(df.index, index=df['job_title'].apply(lambda x: str(x).lower().strip())).drop_duplicates()

print("\nPeta (mapping) judul pekerjaan ke indeks telah dibuat.")


Cosine Similarity Matrix Selesai Dihitung (berdasarkan 'combined_text').
Shape dari Cosine Similarity Matrix: (606, 606)

Peta (mapping) judul pekerjaan ke indeks telah dibuat.


## Recommendation Modelling Function

In [59]:
def recommend_jobs(job_title_input, top_n=5):
    # Normalisasi judul pekerjaan input untuk pencarian (lowercase dan hilangkan spasi ekstra)
    normalized_job_title_input = str(job_title_input).lower().strip()

    # Periksa apakah judul pekerjaan input ada di dalam indeks yang telah dibuat
    if normalized_job_title_input not in indices.index:
        print(f"Peringatan: Judul pekerjaan '{job_title_input}' tidak ditemukan dalam dataset.")
        return {}

    # Dapatkan indeks baris dari judul pekerjaan target dari DataFrame utama
    idx = indices.loc[normalized_job_title_input]

    # Jika ada beberapa entri dengan judul yang sama (karena drop_duplicates), ambil indeks pertama
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    # Dapatkan 'job_id' dari pekerjaan target. Ini penting untuk menghindari rekomendasi duplikasi.
    target_job_id = df.iloc[idx]['job_id']

    # Dapatkan semua skor kemiripan untuk pekerjaan ini dengan semua pekerjaan lain
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Urutkan pekerjaan berdasarkan skor kemiripan dari yang tertinggi ke terendah
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Inisialisasi list untuk menyimpan rekomendasi unik
    recommended_sim_scores = []
    for i, score in sim_scores:
        # Kecualikan pekerjaan jika 'job_id'-nya sama dengan 'job_id' pekerjaan target.
        if df.iloc[i]['job_id'] != target_job_id:
            recommended_sim_scores.append((i, score))

        # Hentikan iterasi setelah mencapai jumlah rekomendasi yang diminta (top_n)
        if len(recommended_sim_scores) >= top_n:
            break

    # Jika tidak ada rekomendasi unik yang ditemukan setelah penyaringan
    if not recommended_sim_scores:
        print(f"Tidak ada rekomendasi unik yang ditemukan untuk '{job_title_input}' selain pekerjaan itu sendiri atau duplikasinya.")
        return {}

    # Ekstrak indeks pekerjaan dan skor kemiripan dari rekomendasi yang sudah difilter
    job_indices = [i[0] for i in recommended_sim_scores]
    similarity_scores = [i[1] for i in recommended_sim_scores]

    # Buat kamus (dictionary) hasil rekomendasi: {Judul Pekerjaan (bersih): Skor Kemiripan}
    recommended_jobs = {}
    for i, score in zip(job_indices, similarity_scores):
        original_job_title = df['job_title'].iloc[i]
        recommended_jobs[original_job_title] = score
    return recommended_jobs

print("\nFungsi `recommend_jobs` telah didefinisikan.")


Fungsi `recommend_jobs` telah didefinisikan.


# Contoh Penggunaan Fungsi Rekomendasi

In [60]:
common_job_titles = df['job_title'].value_counts().head(10).index.tolist()
print(f"\nBeberapa judul pekerjaan yang umum di dataset: {common_job_titles}")

test_job_title = "data analyst" # Contoh: posisi data analyst
if test_job_title.lower() not in [x.lower() for x in common_job_titles] and common_job_titles:
    test_job_title = common_job_titles[0] # Fallback jika judul yang dipilih tidak ada

recommendations = recommend_jobs(test_job_title, top_n=5)

if recommendations:
    print(f"\nRekomendasi untuk '{test_job_title.title()}':")
    for i, (job, score) in enumerate(recommendations.items(), start=1):
        print(f"{i}. {job.title()} — Skor Kemiripan: {score:.4f}")
else:
    print(f"\nTidak ada rekomendasi yang ditemukan untuk '{test_job_title.title()}'.")


Beberapa judul pekerjaan yang umum di dataset: ['data analyst', 'data engineer', 'system analyst', 'it business analyst', 'database administrator', 'data scientist', 'senior data engineer', 'executive business intelligence sas analyst', 'senior engineer engineer data management analytics', 'data scientist hybrid working']

Rekomendasi untuk 'Data Analyst':
1. Ecommerce Data Analyst — Skor Kemiripan: 0.5704


## Simpan Data yang Sudah Diproses Akhir, Vektorisasi, dan Similarity

In [61]:
final_csv_filename_df = 'processed_job_data.csv'
df.to_csv(final_csv_filename_df, index=False)
print(f"\nData DataFrame yang sudah diproses berhasil disimpan ke file CSV: '{final_csv_filename_df}'")

# Simpan DataFrame ke JSON
final_json_filename_df = 'processed_job_data.json'
df.to_json(final_json_filename_df, orient='records', indent=4)
print(f"Data DataFrame yang sudah diproses berhasil disimpan ke file JSON: '{final_json_filename_df}'")


# --- Menyimpan Hasil Vektorisasi (TF-IDF Matrix) ---
tfidf_dense = tfidf_matrix.toarray() # Mengubah sparse matrix menjadi dense array

# Simpan TF-IDF matrix (dense) ke CSV
tfidf_csv_filename = 'tfidf_matrix.csv'
np.savetxt(tfidf_csv_filename, tfidf_dense, delimiter=',')
print(f"\nHasil TF-IDF matrix berhasil disimpan ke file CSV: '{tfidf_csv_filename}'")

# menyimpan dalam format biner NumPy (.npy) yang lebih efisien untuk array
tfidf_npy_filename = 'tfidf_matrix.npy'
np.save(tfidf_npy_filename, tfidf_dense)
print(f"Hasil TF-IDF matrix berhasil disimpan ke file NPY: '{tfidf_npy_filename}'")


# --- Menyimpan Hasil Similarity (Cosine Similarity Matrix) ---
cosine_sim_csv_filename = 'cosine_sim_matrix.csv'
np.savetxt(cosine_sim_csv_filename, cosine_sim_matrix, delimiter=',')
print(f"\nHasil Cosine Similarity matrix berhasil disimpan ke file CSV: '{cosine_sim_csv_filename}'")

# menyimpan dalam format biner NumPy (.npy) yang lebih efisien
cosine_sim_npy_filename = 'cosine_sim_matrix.npy'
np.save(cosine_sim_npy_filename, cosine_sim_matrix)
print(f"Hasil Cosine Similarity matrix berhasil disimpan ke file NPY: '{cosine_sim_npy_filename}'")

print("\nSemua file hasil telah disimpan.")


Data DataFrame yang sudah diproses berhasil disimpan ke file CSV: 'processed_job_data.csv'
Data DataFrame yang sudah diproses berhasil disimpan ke file JSON: 'processed_job_data.json'

Hasil TF-IDF matrix berhasil disimpan ke file CSV: 'tfidf_matrix.csv'
Hasil TF-IDF matrix berhasil disimpan ke file NPY: 'tfidf_matrix.npy'

Hasil Cosine Similarity matrix berhasil disimpan ke file CSV: 'cosine_sim_matrix.csv'
Hasil Cosine Similarity matrix berhasil disimpan ke file NPY: 'cosine_sim_matrix.npy'

Semua file hasil telah disimpan.
